In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/uplift-shift-23/baseline.csv
/kaggle/input/uplift-shift-23/x5-uplift-valid/train_purch/train_purch.csv
/kaggle/input/uplift-shift-23/x5-uplift-valid/data/products.csv
/kaggle/input/uplift-shift-23/x5-uplift-valid/data/clients2.csv
/kaggle/input/uplift-shift-23/x5-uplift-valid/data/train.csv
/kaggle/input/uplift-shift-23/x5-uplift-valid/data/test.csv
/kaggle/input/uplift-shift-23/x5-uplift-valid/test_purch/test_purch.csv


# **Функции для работы**

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder 
from datetime import datetime
from scipy import sparse
from sklearn.metrics import roc_auc_score
from sklearn.base import clone
from sklearn.model_selection import train_test_split

# **Функции загрузки данных и небольшой их предобработки**

In [3]:
#функция загрузки таблицы clients2.csv
def load_clients():
    return pd.read_csv(\
        '/kaggle/input/uplift-shift-23/x5-uplift-valid/data/clients2.csv',
         parse_dates=['first_issue_date', 'first_redeem_date'])

#загрузка + предобработка clients2.csv
def prepare_clients():
    print('Preparing clients...')
    clients = load_clients()
    
    #есть строковый столбец client_id  -> закодируем числами
    client_encoder = LabelEncoder() 
    clients['client_id'] = client_encoder.fit_transform(clients['client_id'])
    print('Clients are ready')
    return clients, client_encoder #для client_encoder.classes_


#загрузка таблицы products.csv
def load_products():
    return pd.read_csv(\
        '/kaggle/input/uplift-shift-23/x5-uplift-valid/data/products.csv')

#загрузка + предобработка products.csv
def prepare_products():
    print('Preparing products...')
    products = load_products()
    product_encoder = LabelEncoder()
    products['product_id'] = product_encoder. \
        fit_transform(products['product_id'])
    products.fillna(-1, inplace=True) #где N/A проставил "-1"
    
    #строковые столбцы кодируем числами
    for col in [
        'level_1', 'level_2', 'level_3', 'level_4',
        'segment_id', 'brand_id', 'vendor_id']:
        products[col] = LabelEncoder().fit_transform(products[col].astype(str))
    print('Products are ready')
    return products, product_encoder


#загрузка таблиц train_purch.csv и test_purch.csv (соединены в одну)
def load_purchases():
    print('Loading purchases...')
    purchases_train = pd.read_csv(\
        '/kaggle/input/uplift-shift-23/x5-uplift-valid/train_purch/train_purch.csv')
    purchases_test = pd.read_csv(\
        '/kaggle/input/uplift-shift-23/x5-uplift-valid/test_purch/test_purch.csv')
    purchases = pd.concat([purchases_train, purchases_test]) #соединение в одну
    print('Purchases are loaded')
    return purchases

#загрузка + предобработка (train_purch.csv + test_purch.csv)
def prepare_purchases(client_encoder,product_encoder):
    print('Preparing purchases...')
    purchases = load_purchases()

    #в ячейки, где N/A проставил "-1"
    print('Handling N/A values...')
    #в 'client_id', 'product_id' ищем пропуски и если есть, то удаляем эти строки
    purchases.dropna(subset=['client_id', 'product_id'],
        how='any', inplace=True)
    #в остальных местах таблицы пропуски заполняем "-1"
    purchases.fillna(-1, inplace=True)

    #кодирование строковых столбцов
    print('Label encoding...')
    #используем кодировки, созданные в функциях prepare_clients и prepare_products 
    purchases['client_id'] = client_encoder.transform(purchases['client_id'])
    purchases['product_id'] = product_encoder.transform(purchases['product_id'])
    for col in ['transaction_id', 'store_id']:
        purchases[col] = LabelEncoder(). \
            fit_transform(purchases[col].astype(str))

    #переводим дату с временем из строкового типа в формат времени и
    #одновременно "переименовываем" столбец transaction_datetime в datetime 
    print('Date and time conversion...')
    purchases['datetime'] = pd.to_datetime(
        purchases['transaction_datetime'], format='%Y-%m-%d %H:%M:%S')
    purchases.drop(columns=['transaction_datetime'], inplace=True)

    print('Purchases are ready')
    return purchases


#загрузка таблицы train.csv
def load_train():
    return pd.read_csv('/kaggle/input/uplift-shift-23/x5-uplift-valid/data/train.csv',
        index_col='client_id')

#загрузка таблицы test.csv
def load_test():
    return pd.read_csv('/kaggle/input/uplift-shift-23/x5-uplift-valid/data/test.csv',
        index_col='client_id')

**Вспомогательные функции**

In [4]:
#когда аггрегирующие функции будут применяться, будут появляться мультииндексы
#эта функция убирает мультииндекс путем соединения индексов знаком "_"
def drop_column_multi_index_inplace(df):
    df.columns = ['_'.join(t) for t in df.columns]
    

#эти функции применял для фичей, связанных со временем покупки (утро/день, пн/вт/...)  
def make_sum_csr(df,index_col,value_col,col_to_sum):
    print(df[col_to_sum].values.shape)
    print(df[index_col].values.shape)
    print(df[value_col].values.shape)
    coo = sparse.coo_matrix((df[col_to_sum].values,(df[index_col].values,df[value_col].values)))
    csr = coo.tocsr(copy=False)
    return csr
        
def make_count_csr(df,index_col,value_col):
    col_to_sum_name = '__col_to_sum__'
    df['__col_to_sum__'] = 1
    csr = make_sum_csr(df,index_col=index_col,value_col=value_col,col_to_sum=col_to_sum_name)
    df.drop(columns=col_to_sum_name, inplace=True)
    return csr    

# **функции для создания фичей**

**функция создания фичей, связанных с клиентами**

In [5]:
SECONDS_IN_DAY = 60 * 60 * 24
RANDOM_STATE = 1

def make_client_features(clients):
    print('Preparing features...')
    #самая ранняя дата получения карты среди клиентов
    min_datetime = clients['first_issue_date'].min() 
    #число дней с min_datetime до даты получения карты для каждого клиента
    days_from_min_to_issue = (
        (clients['first_issue_date'] - min_datetime).dt.total_seconds() /
            SECONDS_IN_DAY
    ).values
    #число дней с min_datetime до first_redeem_date
    days_from_min_to_redeem = (
            (clients['first_redeem_date'] - min_datetime).dt.total_seconds() /
            SECONDS_IN_DAY
    ).values

    #возраст клиентов есть 1852 и -7491?!! => заменяем на числа "-2" и "-3"
    age = clients['age'].values
    age[age < 0] = -2
    age[age > 100] = -3
    
    #объединение фичей
    print('Combining features')
    gender = clients['gender'].values
    features = pd.DataFrame({
        'client_id': clients['client_id'].values,
        'gender_M': (gender == 'M').astype(int),
        'gender_F': (gender == 'F').astype(int),
        'gender_U': (gender == 'U').astype(int),
        'age': age,
        'days_from_min_to_issue': days_from_min_to_issue,
        'days_from_min_to_redeem': days_from_min_to_redeem,
        'issue_redeem_delay': days_from_min_to_redeem - days_from_min_to_issue})
    #если вдруг остались пропуски => на "-1"
    features = features.fillna(-1)
    print(f'Client features are created. Shape = {features.shape}')
    return features

**функция создания фичей, связанных с продуктами**

In [6]:
def make_product_features(products,purchases):
    #merge таблиц purchases и products ("inner")
    print('Creating purchases-products matrix')
    purchases_products = pd.merge(purchases,products,on='product_id')
    print('Purchases-products matrix is ready')

    del purchases
    del products

    print('Creating usual features')
    usual_features = make_usual_features(purchases_products)

    print(f'Product features are created. Shape = {usual_features.shape}')
    return usual_features     


#обычные фичи для клиента по покупкам(число уникальных, медиана, макс, мин, сумма)
def make_usual_features(purchases_products):
    #purchases_products - появляется в make_product_features, где эта функция 
    #и применяется
    pp_gb = purchases_products.groupby('client_id')
    usual_features = pp_gb.agg(
        {
            'netto': ['median', 'max', 'sum'],
            'is_own_trademark': ['sum', 'mean'],
            'is_alcohol': ['sum', 'mean'],
            'level_1': ['nunique'],
            'level_2': ['nunique'],
            'level_3': ['nunique'],
            'level_4': ['nunique'],
            'segment_id': ['nunique'],
            'brand_id': ['nunique'],
            'vendor_id': ['nunique']})
    drop_column_multi_index_inplace(usual_features)
    usual_features.reset_index(inplace=True)
    return usual_features

**фичи, связанные с покупками**

In [7]:
ORDER_COLUMNS = [
    'transaction_id',
    'datetime',
    'regular_points_received',
    'express_points_received',
    'regular_points_spent',
    'express_points_spent',
    'purchase_sum',
    'store_id'
]

FLOAT32_MAX = np.finfo(np.float32).max
TODAY_DATETIME =datetime(2019, 3, 20)
POINT_TYPES = ('regular', 'express')
POINT_EVENT_TYPES = ('spent', 'received')

#фичи по покупкам за последние n_days (пробовал добавить к фичам, но score 
#становился меньше)
def make_purchase_features_for_last_days(purchases,n_days):
    print(f'Creating purchase features for last {n_days} days...')
    cutoff = TODAY_DATETIME - timedelta(days=n_days)
    purchases_last = purchases[purchases['datetime'] >= cutoff]
    purchase_last_features = make_purchase_features(purchases_last)
    purchase_last_features = purchase_last_features.rename(
        columns=lambda x: x+f'_for_last_{n_days}_days' if x!='client_id' else x)
    print(f'Purchase features for last {n_days} days are created')
    return purchase_last_features


#фичи по покупкам
def make_purchase_features(purchases):
    print('Creating purchase features...')
    n_clients = purchases['client_id'].nunique()

    print('Creating really purchase features...')
    #make_really_purchase_features - ниже
    purchase_features = make_really_purchase_features(purchases)
    print('Really purchase features are created')

    print('Creating small product features...')
    #make_small_product_features - ниже
    product_features = make_small_product_features(purchases)
    print('Small product features are created')

    print('Preparing orders table...')

    orders = purchases.reindex(columns=['client_id'] + ORDER_COLUMNS)
    del purchases
    orders.drop_duplicates(inplace=True)
    print(f'Orders table is ready. Orders: {len(orders)}')

    print('Creating order features...')
    #make_order_features - ниже
    order_features = make_order_features(orders)
    print('Order features are created')

    print('Creating store features...')
    #make_store_features - ниже
    store_features = make_store_features(orders)
    print('Store features are created')

    print('Creating order interval features...')
    #make_order_interval_features - ниже
    order_interval_features = make_order_interval_features(orders)
    print('Order interval features are created')

    print('Creating features for orders with express points spent ...')
    #make_features_for_orders_with_express_points_spent - ниже
    orders_with_express_points_spent_features = \
        make_features_for_orders_with_express_points_spent(orders)
    print('Features for orders with express points spent are created')

    #соединение фич
    features = (
        purchase_features
        .merge(order_features, on='client_id')
        .merge(product_features, on='client_id')
        .merge(store_features, on='client_id')
        .merge(order_interval_features, on='client_id')
        .merge(orders_with_express_points_spent_features, on='client_id')
    )

    #проверка на равенство числа строк в таблицах
    assert len(features) == n_clients, \
        f'n_clients = {n_clients} but len(features) = {len(features)}'

    #добавление еще фич
    features['days_from_last_order_share'] = \
        features['days_from_last_order'] / features['orders_interval_median']

    features['most_popular_store_share'] = (
        features['store_transaction_id_count_max'] /
        features['transaction_id_count']
    )

    features['ratio_days_from_last_order_eps_to_median_interval_eps'] = (
        features['days_from_last_express_points_spent'] /
        features['orders_interval_median_eps']
    )

    features['ratio_mean_purchase_sum_eps_to_mean_purchase_sum'] = (
        features['median_purchase_sum_eps'] /
        features['purchase_sum_median']
    )

    print(f'Purchase features are created. Shape = {features.shape}')
    return features

**функции, которые вызываются в функции make_purchase_features(purchases)**

In [8]:
def make_really_purchase_features(purchases):
    simple_purchases = purchases.reindex(
        columns=['client_id', 'product_id', 'trn_sum_from_iss']
    )
    prices_bounds = [0, 98, 195, 490, 950, 1900, 4400, FLOAT32_MAX]
    agg_dict = {}
    #принадлежность trn_sum_from_iss промежуткам вида [lower_bound, upper_bound)
    for i, lower_bound in enumerate(prices_bounds[:-1]):
        upper_bound = prices_bounds[i + 1]
        name = f'price_from_{lower_bound}'
        simple_purchases[name] = (
            (simple_purchases['trn_sum_from_iss'] >= lower_bound) &
            (simple_purchases['trn_sum_from_iss'] < upper_bound)
        ).astype(int)
        agg_dict[name] = ['sum', 'mean']

    agg_dict.update(
        {
            'trn_sum_from_iss': ['median'],  
            'product_id': ['count', 'nunique']
        }
    )
    simple_features = simple_purchases.groupby('client_id').agg(agg_dict)
    drop_column_multi_index_inplace(simple_features)
    simple_features.reset_index(inplace=True)

    p_gb = purchases.groupby(['client_id', 'transaction_id'])
    purchase_agg = p_gb.agg(
        {
            'product_id': ['count'],
            'product_quantity': ['max']
        }
    )
    drop_column_multi_index_inplace(purchase_agg)
    purchase_agg.reset_index(inplace=True)
    o_gb = purchase_agg.groupby('client_id')
    complex_features = o_gb.agg(
        {
            'product_id_count': ['mean', 'median'],
            'product_quantity_max': ['mean', 'median']
        }
    )
    drop_column_multi_index_inplace(complex_features)
    complex_features.reset_index(inplace=True)
    features = pd.merge(
        simple_features,
        complex_features,
        on='client_id'
    )
    return features



def make_small_product_features(purchases):
    cl_pr_gb = purchases.groupby(['client_id', 'product_id'])
    product_agg = cl_pr_gb.agg({
        'product_quantity': ['sum']})

    drop_column_multi_index_inplace(product_agg)
    product_agg.reset_index(inplace=True)

    cl_gb = product_agg.groupby(['client_id'])
    features = cl_gb.agg({'product_quantity_sum': ['max']})

    drop_column_multi_index_inplace(features)
    features.reset_index(inplace=True)

    return features



def make_order_features(orders):
    o_gb = orders.groupby('client_id')

    agg_dict = {
            'transaction_id': ['count'],  
            'regular_points_received': ['sum', 'max', 'median'],
            'express_points_received': ['sum', 'max', 'median'],
            'regular_points_spent': ['sum', 'min', 'median'],
            'express_points_spent': ['sum', 'min', 'median'],
            'purchase_sum': ['sum', 'max', 'median'],
            'store_id': ['nunique'],  
            'datetime': ['max'] 
        }

    #  regular/express points потрачены/получены?
    for points_type in POINT_TYPES:
        for event_type in POINT_EVENT_TYPES:
            col_name = f'{points_type}_points_{event_type}'
            new_col_name = f'is_{points_type}_points_{event_type}'
            orders[new_col_name] = (orders[col_name] != 0).astype(int)
            agg_dict[new_col_name] = ['sum']

    features = o_gb.agg(agg_dict)
    drop_column_multi_index_inplace(features)
    features.reset_index(inplace=True)

    features['days_from_last_order'] = (
        TODAY_DATETIME - features['datetime_max']
    ).dt.total_seconds() // SECONDS_IN_DAY
    features.drop(columns=['datetime_max'], inplace=True)

    # отношение потраченных regular/express points ко всем транзакциям
    for points_type in POINT_TYPES:
        for event_type in POINT_EVENT_TYPES:
            col_name = f'is_{points_type}_points_{event_type}_sum'
            new_col_name = f'proportion_count_{points_type}_points_{event_type}'
            features[new_col_name] = (
                    features[col_name] / features['transaction_id_count']
            )

    express_col = f'is_express_points_spent_sum'
    regular_col = f'is_regular_points_spent_sum'
    new_col_name = f'ratio_count_express_to_regular_points_spent'
    features[new_col_name] = (
            features[express_col] / features[regular_col]
    ).replace(np.inf, FLOAT32_MAX) #если деление на ноль, то замена inf на число

    for points_type in POINT_TYPES:
        spent_col = f'is_{points_type}_points_spent_sum'
        received_col = f'is_{points_type}_points_received_sum'
        new_col_name = f'ratio_count_{points_type}_points_spent_to_received'
        features[new_col_name] = (
                features[spent_col] / features[received_col]
        ).replace(np.inf, 1000)


    for points_type in POINT_TYPES:
        spent_col = f'{points_type}_points_spent_sum'
        orders_sum_col = f'purchase_sum_sum'
        new_col_name = f'ratio_sum_{points_type}_points_spent_to_purchases_sum'
        features[new_col_name] = features[spent_col] / features[orders_sum_col]

    new_col_name = f'ratio_sum_express_points_spent_to_sum_regular_points_spent'
    regular_col = f'regular_points_spent_sum'
    express_col = f'express_points_spent_sum'
    features[new_col_name] = features[express_col] / features[regular_col]

    return features



def make_store_features(orders):
    cl_st_gb = orders.groupby(['client_id', 'store_id'])
    store_agg = cl_st_gb.agg({
        'transaction_id': ['count']})

    drop_column_multi_index_inplace(store_agg)
    store_agg.reset_index(inplace=True)

    cl_gb = store_agg.groupby(['client_id'])
    simple_features = cl_gb.agg(
        {
            'transaction_id_count': ['max', 'mean', 'median']
        }
    )

    drop_column_multi_index_inplace(simple_features)
    simple_features.reset_index(inplace=True)
    simple_features.columns = (
        ['client_id'] +
        [
            f'store_{col}'
            for col in simple_features.columns[1:]
        ]
    )

    return simple_features  



def make_order_interval_features(orders):
    orders = orders.sort_values(['client_id', 'datetime'])

    last_order_client = orders['client_id'].shift(1)
    is_same_client = last_order_client == orders['client_id']
    orders['last_order_datetime'] = orders['datetime'].shift(1)

    orders['orders_interval'] = np.nan
    orders.loc[is_same_client, 'orders_interval'] = (
        orders.loc[is_same_client, 'datetime'] -
        orders.loc[is_same_client, 'last_order_datetime']
    ).dt.total_seconds() / SECONDS_IN_DAY

    cl_gb = orders.groupby('client_id', sort=False)
    features = cl_gb.agg(
        {'orders_interval': ['mean',  'median','std',  'min','max','last']}
    )
    drop_column_multi_index_inplace(features)
    features.reset_index(inplace=True)
    features.fillna(-3, inplace=True)

    return features



def make_features_for_orders_with_express_points_spent(orders):
    orders_with_eps = orders.loc[orders['express_points_spent'] != 0]

    o_gb = orders_with_eps.groupby(['client_id'])
    features = o_gb.agg(
        {'purchase_sum': ['median'], 'datetime': ['max']}
    )
    drop_column_multi_index_inplace(features)
    features.reset_index(inplace=True)
    features['days_from_last_express_points_spent'] = (
            TODAY_DATETIME - features['datetime_max']
    ).dt.days
    features.drop(columns=['datetime_max'], inplace=True)
    features.rename(
        columns={
            'purchase_sum_median': 'median_purchase_sum_eps'
        },
        inplace=True)

    # make_order_interval_features - выше
    order_int_features = make_order_interval_features(orders_with_eps)
    renamings = {
        col: f'{col}_eps'
        for col in order_int_features
        if col != 'client_id'
    }
    order_int_features.rename(columns=renamings, inplace=True)

    features = pd.merge(
        features,
        order_int_features,
        on='client_id')

    features = features.merge(
        pd.Series(orders['client_id'].unique(), name='client_id'),
        how='right')

    return features

**функция для создания фичей, связанный со временем покупки(утро/день..., пн/вт...)**

In [9]:
WEEK_DAYS = [
    'Monday',
    'Tuesday',
    'Wednesday',
    'Thursday',
    'Friday',
    'Saturday',
    'Sunday'
]
TIME_LABELS = ['Night', 'Morning', 'Afternoon', 'Evening']

def make_time_features(orders):
    # np.unique возвращает отсортированный массив
    client_ids = np.unique(orders['client_id'].values)

    orders['weekday'] = np.array(WEEK_DAYS)[
        orders['datetime'].dt.dayofweek.values
    ]

    time_bins = [-1, 6, 11, 18, 24] #утро/день/вечер/ночь

    orders['part_of_day'] = pd.cut(
        orders['datetime'].dt.hour,
        bins=time_bins,
        labels=TIME_LABELS
    ).astype(str)

    time_part_encoder = LabelEncoder()
    orders['part_of_day'] = time_part_encoder.fit_transform(orders['part_of_day'])

    time_part_columns_name = time_part_encoder.inverse_transform(
        np.arange(len(time_part_encoder.classes_))
    )

    time_part_cols = make_count_csr(orders,index_col='client_id',value_col='part_of_day')[client_ids, :]  # drop empty rows

    time_part_cols = pd.DataFrame(
        time_part_cols.toarray(),
        columns=time_part_columns_name)
    time_part_cols['client_id'] = client_ids

    weekday_encoder = LabelEncoder()
    orders['weekday'] = weekday_encoder.fit_transform(orders['weekday'])

    weekday_column_names = weekday_encoder.inverse_transform(
        np.arange(len(weekday_encoder.classes_))
    )
    weekday_cols = make_count_csr(
        orders,
        index_col='client_id',
        value_col='weekday')[client_ids, :]  
    weekday_cols = pd.DataFrame(
        weekday_cols.toarray(),
        columns=weekday_column_names)
    weekday_cols['client_id'] = client_ids

    time_part_features = pd.merge(
        left=time_part_cols,
        right=weekday_cols,
        on='client_id')
    time_part_features.columns = [
        f'{col}_orders_count' if col != 'client_id' else col
        for col in time_part_features.columns
    ]

    return time_part_features

**функции для обучения модели и проверки ее качества**

In [10]:
#функция для обучения модели
def uplift_fit(model, X_train, treatment_train, target_train):
    z = make_z(treatment_train, target_train)
    model = clone(model)
    model.fit(X_train, z)
    return model

#функция для предсказания вероятностей/подсчета аплифта
def uplift_predict(model, X_test, z=True):
    predict_z = model.predict_proba(X_test)[:, 1]
    uplift = calc_uplift(predict_z)
    if z: return predict_z
    else: return uplift
    

#функции, которые вызываются выше    
def make_z(treatment, target):
    y = target
    w = treatment
    z = y * w + (1 - y) * (1 - w) 
    return z 

def calc_uplift(prediction):
    uplift = 2 * prediction - 1
    return uplift


**функции для расчета score модели (uplift, roc_auc**)

In [11]:
def score_uplift(prediction,treatment,target,rate = 0.3):
    order = np.argsort(-prediction)
    treatment_n = int((treatment == 1).sum() * rate)
    treatment_p = target[order][treatment[order] == 1][:treatment_n].mean()
    control_n = int((treatment == 0).sum() * rate)
    control_p = target[order][treatment[order] == 0][:control_n].mean()
    score = treatment_p - control_p
    return score


def score_roc_auc(prediction,treatment,target):
    y_true = make_z(treatment, target)
    score = roc_auc_score(y_true, prediction)
    return score


def uplift_metrics(prediction,treatment,target,rate_for_uplift = 0.3):
    scores = {
        'roc_auc': score_roc_auc(prediction, treatment, target),
        'uplift': score_uplift(prediction, treatment, target, rate_for_uplift)
    }
    return scores

**Функция создания таблицы с фичами**

In [12]:
def prepare_features():
    print('Loading data...')
    clients, client_encoder = prepare_clients()
    products, product_encoder = prepare_products()
    purchases = prepare_purchases(client_encoder, product_encoder)
    
    del product_encoder
    print('Data is loaded')

    print('Preparing features...')
    purchase_features = make_purchase_features(purchases)

    purchases_ids = purchases.reindex(columns=['client_id', 'product_id'])
    
    orders = purchases.reindex(columns=['client_id'] + ORDER_COLUMNS)
    orders.drop_duplicates(inplace=True)
    time_features=make_time_features(orders)
    del purchases
    product_features = make_product_features(products, purchases_ids)
    del purchases_ids

    client_features = make_client_features(clients)

    print('Combining features...')
    features = (
        client_features
            .merge(purchase_features, on='client_id', how='left')
            .merge(product_features, on='client_id', how='left')
            .merge(time_features, on='client_id', how='left')
    )
    del client_features
    del purchase_features
    del product_features

    features.fillna(-2, inplace=True)

    features['client_id'] = client_encoder.inverse_transform(features['client_id'])
    del client_encoder

    print('Features are ready')

    return features


**Функция сохранения результатов**

In [13]:
def save_submission(indices_test, test_pred, filename):
    df_submission = pd.DataFrame({'pred': test_pred}, index=indices_test)
    df_submission.to_csv(filename)

**Создание таблицы фичей**

In [14]:
features = prepare_features()
print(f'Features shape: {features.shape}')
features.set_index('client_id', inplace=True)
features

Loading data...
Preparing clients...
Clients are ready
Preparing products...
Products are ready
Preparing purchases...
Loading purchases...
Purchases are loaded
Handling N/A values...
Label encoding...
Date and time conversion...
Purchases are ready
Data is loaded
Preparing features...
Creating purchase features...
Creating really purchase features...
Really purchase features are created
Creating small product features...
Small product features are created
Preparing orders table...
Orders table is ready. Orders: 4024949
Creating order features...
Order features are created
Creating store features...
Store features are created
Creating order interval features...
Order interval features are created
Creating features for orders with express points spent ...
Features for orders with express points spent are created
Purchase features are created. Shape = (200039, 76)
(4024949,)
(4024949,)
(4024949,)
(4024949,)
(4024949,)
(4024949,)
Creating purchases-products matrix
Purchases-products matri

,gender_M,gender_F,gender_U,age,days_from_min_to_issue,days_from_min_to_redeem,issue_redeem_delay,price_from_0_sum,price_from_0_mean,price_from_98_sum,...,Evening_orders_count,Morning_orders_count,Night_orders_count,Friday_orders_count,Monday_orders_count,Saturday_orders_count,Sunday_orders_count,Thursday_orders_count,Tuesday_orders_count,Wednesday_orders_count
client_id,,,,,,,,,,,,,,,,,,,,,
000012768d,0,0,1,45,122.886458,275.045706,152.159248,48,0.923077,4,...,0,3,0,1,0,1,1,1,0,0
000036f903,0,1,0,72,5.812558,18.759468,12.946910,141,0.870370,19,...,0,31,0,3,3,1,9,3,7,6
00010925a5,0,0,1,83,475.914711,527.908692,51.993981,61,0.782051,14,...,0,13,1,1,5,1,2,2,6,1
0001f552b0,0,1,0,33,87.039120,510.774618,423.735498,70,0.813953,11,...,0,8,0,1,3,4,2,3,0,2
00020e7b18,0,0,1,73,236.720451,280.976238,44.255787,186,0.683824,60,...,0,15,1,6,0,0,0,3,3,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fffe0abb97,0,1,0,35,236.605972,312.626273,76.020301,27,0.710526,10,...,0,5,3,5,0,1,2,0,1,0
fffe0ed719,0,0,1,69,163.603542,251.851319,88.247778,143,0.831395,28,...,0,4,0,4,4,4,4,8,2,4
fffea1204c,0,1,0,73,301.941192,341.943160,40.001968,46,0.754098,11,...,0,1,0,4,0,2,1,2,2,6


# **Подготовка датасетов**

In [15]:
train = load_train()
test = load_test()
indices_train = train.index #клиенты в тренировочном наборе
indices_test = test.index  #клиенты в тестовом наборе

X_train = features.loc[indices_train]
treatment_train = train.loc[indices_train, 'treatment_flg'].values
target_train = train.loc[indices_train, 'purchased'].values

X_test = features.loc[indices_test]

indices_learn, indices_valid = train_test_split(train.index,test_size=0.2,random_state=RANDOM_STATE + 1)

X_learn = features.loc[indices_learn]
treatment_learn = train.loc[indices_learn, 'treatment_flg'].values
target_learn = train.loc[indices_learn, 'purchased'].values

X_valid = features.loc[indices_valid]
treatment_valid = train.loc[indices_valid, 'treatment_flg'].values
target_valid = train.loc[indices_valid, 'purchased'].values
print('Data sets prepared')

Data sets prepared


**Как подбирались гиперпараметры модели**

Я не успел подготовить ноутбук, где отражен весь мой порядок действий (имеется в 
виду код, который выполнялся) по поиску гиперпараметров модели, потому что подбор 
проводился вручную, но обдуманно и как только получал какой-либо результат сразу же 
переписывал функции, где менял диапазоны значений параметров. Но здесь напишу словами как производился подбор гиперпараметров.

1) Я решил использовать light gradient boost machine classifier
(LGBMClassifier). В нем множество гиперпараметров. Сначала я попытался получить 
хоть какую-то модель, поэтому интуитивно взял следующие значения параметров. 
boosting_type выбрал rf, то есть random forest, т.к. с ним уже сталкивался. num_leaves 
по умолчанию равно 31, я решил взять немного побольше - 40. max_depth = 3 - глубина 
леса = 2 это мало почти всегда, поэтому попробовал 3, learning_rate=0.001 - решил
выбрать достаточно малую скорость обучения. n_estimators=15000 - часто чем больше, тем 
лучше (но слишком много тоже плохо). Параметр min_child_samples = 20. objective='binary' - для LGBMClassifier.  В дополнение я добавил еще несколько 
гиперпараметров: is_unbalance=True - нужно указывать если выбрал objective='binary', 
max_bin=100 - для борьбы с переобучением, bagging_freq=1 - чтобы на каждой итерации 
был bagging, попробовал  bagging_fraction=0.5 (взял среднее между минимальным и 
максимальным) для увеличения скорости обучения и борьбы с переобучением. Остальные 
гиперпараметры оставил такими, как по умолчанию.
В итоге получил score: 0.04409 - что весьма неплохо.

2) Далее решил увеличить значения параметра 'max_depth', прогнал для 3, 4, 5, 6 и 
увидел, что для 4 результат стал лучше во всех смыслах, а начиная с 5 learning score был больше, а валидационный меньше, что является признаком 
переобучения. Поэтому я далее глубину не выставлял больше 4.

3) Попробовал поварьировать значение параметра скорости обучения при прочих неизменных параметрах- в результате качество модели не менялось => зафиксировал для всех последующих запусков кода learning_rate=0.001

4) Создал функцию find_best_params, в которой поварьировал параметры 
'max_depth' = 3 и 4, 'n_estimators'= 15000, 20000,
'min_child_samples' = 15, 20, 25, 'num_leaves'=30, 35, 40, 'max_bin'=50, 75, 100
Результаты заносил в df. Анализируя данные пришел к выводу о том, что гиперпараметр 
num_leaves мало влияет на качество модели, поэтому в дальнейшем использовал значение 40. Среди рассмотренных наборов параметров лучшим оказался следующий:
max_depth=4, n_estimators=20000, min_child_samples=15, num_leaves=40, max_bin=100.
Остальные гиперпараметры остались без изменения как в пункте 1)
При таких параметрах  score: 0.04869 - стало лучше.

5) Далее зафиксировал все параметры, кроме n_estimators и min_child_samples: обучал модели с разными значениями этих гиперпараметров. В среднем наблюдается, что с ростом числа деревьев в лесу, растет roc_auc, эта метрика также растет с уменьшением min_child_samples (наиболее оптимальное значение min_child_samples=5)

6) Затем попытался добавить регуляризацию. Поварьировал параметры reg_alpha и reg_lambda. Лучшее значение roc_auc для валидационных данных получилось при 
reg_alpha=reg_lambda=0.01
В результате score:0.04875 - чуть-чуть улучшился

7) Понял, что прогресса почти нет и добавил новые фичи, связанные со временем совершения покупки (утро/день..., пн/вт/ср...) и запустил обучение модели, с параметрами, как в пункте 6).
В результате score: 0.04957 - еще лучше

8) Затем оставив все гиперпараметры такими же, я уменьшил долю тестовых данных, т. е. test_size уменьшил с 0.3 до 0.2.
В итоге score: 0.05036 (пробовал еще больше уменьшить долю валидационных данных, 
но score уменьшался в таком случае - видимо нужно другие гиперпараметры подбирать)
   

**Классификатор**

In [16]:
#так подбирались гиперпараметры в пунктах 4-5

# from lightgbm import LGBMClassifier
# parameters={
#     'max_depth': [4], #4
#     'n_estimators': range(10000, 25000, 5000), 
#     'min_child_samples': range(15,25,5), 
#     'num_leaves': [40], 
#     'max_bin': range(45, 100, 15), 
# }

# #создание классификатора с заданными параметрами
# def classificator(max_depth, n_estimators, min_child_samples, num_leaves, max_bin):
#     clf = LGBMClassifier(
#         boosting_type='rf', # ‘gbdt’, ‘dart’, ‘goss’
#         n_estimators=n_estimators,
#         num_leaves=num_leaves,
#         max_depth=max_depth,
#         learning_rate=0.001,
#         random_state=RANDOM_STATE,
#         bagging_freq=1,
#         bagging_fraction=0.5,
#         importance_type='split',
#         is_unbalance=True,
#         min_child_samples=min_child_samples,
#         min_child_weight=0.001,
#         min_split_gain=0.0,
#         objective='binary',
#         reg_alpha=0.0,
#         reg_lambda=0.0,
#         verbose=-1,
#         subsample=1.0,
#         subsample_freq=0,
#         max_bin=max_bin           
#     )
#     return clf


# #поиск score на валидационных и тренировочных данных для данного классификатора
# def find_scores(max_depth, n_estimators, min_child_samples, num_leaves, max_bin):
#     clf_0=classificator(max_depth, n_estimators, min_child_samples, num_leaves, max_bin)
#     print('fitting...')
#     clf = uplift_fit(clf_0, X_learn, treatment_learn, target_learn) #обучение
#     print('successful!')
#     learn_pred = uplift_predict(clf, X_learn)
#     learn_scores = uplift_metrics(learn_pred, treatment_learn, target_learn)
#     valid_pred = uplift_predict(clf, X_valid)
#     valid_scores = uplift_metrics(valid_pred, treatment_valid, target_valid)
#     print('scores recieved')
#     return learn_scores, valid_scores
 
# #занесение результатов в таблицу
# answers = pd.DataFrame({
#     'max_depth': [],
#     'n_estimators': [],
#     'min_child_samples': [],
#     'num_leaves': [],
#     'max_bin': [],
#     'learn_roc_auc': [],
#     'learn_uplift': [],
#     'valid_roc_auc': [],
#     'valid_uplift': [],
# })
# def find_best_params(answers):
#     step_num=0 #какая по счету строчка таблицы answers будет заполняться
#     depth=4
#     num_leaves=40
# #     min_child_samples=15
# #     max_bin=100
#     for n_estimators in parameters['n_estimators']:
#         for min_child_samples in parameters['min_child_samples']:
#             for max_bin in parameters['max_bin']:
#                 step_num=step_num+1
#                 print('step_num', step_num, 'out of', 72)
#                 learn_scores, valid_scores=find_scores(depth, n_estimators, \
#                                     min_child_samples, num_leaves, max_bin)
#                 learn_roc_auc, learn_uplift = learn_scores['roc_auc'], \
#                                               learn_scores['uplift']
#                 valid_roc_auc, valid_uplift = valid_scores['roc_auc'], \
#                                               valid_scores['uplift']
#                 #добавление строки с результатами
#                 answers.loc[len(answers.index)]=[depth, n_estimators, \
#                         min_child_samples,num_leaves, max_bin, learn_roc_auc,\
#                         learn_uplift, valid_roc_auc, valid_uplift]
#                 #вывод текущей строки
#                 print([depth, n_estimators, min_child_samples, num_leaves, \
#                        max_bin, learn_roc_auc, learn_uplift, valid_roc_auc, \
#                        valid_uplift])

#     return answers

In [17]:
#подбор параметров для регуляризации
from lightgbm import LGBMClassifier
parameters={
    'reg_alpha': [0.0, 0.005, 0.01, 0.05, 0.1, 0.2], #6 штук
    'reg_lambda': [0.0, 0.005, 0.01, 0.05, 0.1, 0.2], #6 штук
}


#создание классификатора с заданными параметрами
def classificator(reg_alpha, reg_lambda):
    clf = LGBMClassifier(
        boosting_type='rf', # ‘gbdt’, ‘dart’, ‘goss’
        n_estimators=20000,
        num_leaves=40,
        max_depth=4,
        learning_rate=0.001,
        random_state=RANDOM_STATE,
        bagging_freq=1,
        bagging_fraction=0.5,
        importance_type='split',
        is_unbalance=True,
        min_child_samples=5,
        min_child_weight=0.001,
        min_split_gain=0.0,
        objective='binary',
        reg_alpha=reg_alpha,
        reg_lambda=reg_lambda,
        verbose=-1,
        subsample=1.0,
        subsample_freq=0,
        max_bin=100           
    )
    return clf


#поиск score на валидационных и тренировочных данных для данного классификатора
def find_scores(reg_alpha, reg_lambda):
    clf_0=classificator(reg_alpha, reg_lambda)
    print('fitting...')
    clf = uplift_fit(clf_0, X_learn, treatment_learn, target_learn) #обучение
    print('successful!')
    learn_pred = uplift_predict(clf, X_learn)
    learn_scores = uplift_metrics(learn_pred, treatment_learn, target_learn)
    valid_pred = uplift_predict(clf, X_valid)
    valid_scores = uplift_metrics(valid_pred, treatment_valid, target_valid)
    print('scores recieved')
    return learn_scores, valid_scores
 
#занесение результатов в таблицу
answers = pd.DataFrame({
    'reg_alpha': [], 
    'reg_lambda': [],
    'learn_roc_auc': [],
    'learn_uplift': [],
    'valid_roc_auc': [],
    'valid_uplift': [],
})
def find_best_params(answers):
    step_num=0 #какая по счету строчка таблицы answers будет заполняться
#     depth=4
#     num_leaves=40
#     max_bin=100
#     min_child_samples=5

    for reg_alpha in parameters['reg_alpha']:
        for reg_lambda in parameters['reg_lambda']:
            step_num=step_num+1
            print('step_num', step_num, 'out of', 36)
            learn_scores, valid_scores=find_scores(reg_alpha, reg_lambda)
            learn_roc_auc, learn_uplift = learn_scores['roc_auc'], \
                                          learn_scores['uplift']
            valid_roc_auc, valid_uplift = valid_scores['roc_auc'], \
                                          valid_scores['uplift']
            #добавление строки с результатами
            answers.loc[len(answers.index)]=[reg_alpha, reg_lambda, learn_roc_auc,\
                    learn_uplift, valid_roc_auc, valid_uplift]
            #вывод текущей строки
            print([reg_alpha, reg_lambda, learn_roc_auc, learn_uplift, valid_roc_auc,\
                   valid_uplift])

    return answers
    

**Сохранение результатов для различных гиперпараметров (закомментировал, т.к. несколько часов вызывается)**

In [18]:
# results=find_best_params(answers)
# results.to_csv("find_best_clf.csv", index=False)

**Поиск лучших гиперпараметров (используются результаты предыдущей ячейки)**

In [19]:
# results= pd.read_csv('./find_best_clf.csv')
# TYPES_OF_BEST=['learn_roc_auc', 'learn_uplift', 'valid_roc_auc', 'valid_uplift']

# #поиск строк где лучшее (с выбором какой score нужен для поиска лучшего показателя)
# def find_best_params(type_of_best, df):
#     name=type_of_best
#     max_value=df[name].max()
#     indices_where_max_value=df[df[name] == max_value].index
#     return df.iloc[indices_where_max_value]
    
# #вывод всех лучших показателей
# def print_all_best_results(TYPES_OF_BEST, df):
#     for type_of_best in TYPES_OF_BEST:
#         print(f'for {type_of_best}:')
#         print(find_best_params(type_of_best, df))


# print_all_best_results(TYPES_OF_BEST, results)   
# #но больше всего интересует случай, с наилучшим значением 'valid_roc_auc'

**Применение лучшего классификатора на тестовых данных**

In [20]:
clf_=classificator(0.01,0.01)
print('Build model for learn data set...')
clf = uplift_fit(clf_, X_learn, treatment_learn, target_learn)
print('Model is ready')

learn_pred = uplift_predict(clf, X_learn)
learn_scores = uplift_metrics(learn_pred, treatment_learn, target_learn)
print(f'Learn scores: {learn_scores}')
valid_pred = uplift_predict(clf, X_valid)
valid_scores = uplift_metrics(valid_pred, treatment_valid, target_valid)
print(f'Valid scores: {valid_scores}')



Build model for learn data set...
[LightGBM] [Warning] bagging_fraction is set=0.5, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
Model is ready
Learn scores: {'roc_auc': 0.5389854337841111, 'uplift': 0.11175936158456246}
Valid scores: {'roc_auc': 0.5238371510230764, 'uplift': 0.0735946099984085}


In [21]:
#сохранение результата для отправки
test_pred = uplift_predict(clf, X_test, z = True)
print('Saving submission...')
save_submission(indices_test,test_pred,'submission_16.csv')
print('Submission is ready')

Saving submission...
Submission is ready


**Важность фичей**

In [22]:
import lightgbm
importances =clf.feature_importances_
feature_names=clf.feature_name_
clf_importances = pd.DataFrame({'feature_name': feature_names, 'importance':importances})
clf_importances=clf_importances.sort_values(by='importance', ascending=True).head(20)

clf_importances

#можно попробовать обучать классификатор после удаления нескольких наиболее 
#плохих фичей

,feature_name,importance
34,express_points_received_median,0
19,price_from_4400_sum,0
48,is_express_points_received_sum,1
33,express_points_received_max,1
20,price_from_4400_mean,4
56,ratio_count_express_points_spent_to_received,12
32,express_points_received_sum,19
17,price_from_1900_sum,38
53,proportion_count_express_points_received,54
15,price_from_950_sum,124
